<a href="https://colab.research.google.com/github/jingxiangwu/games/blob/main/three_way_duel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem statement

Alice, Bob, and Carol arrange a three-way duel.

Each of the three has a gun and plenty of ammunition.

If anyone takes a shot at one of his opponents, there is a fixed probability of killing whomever he shoots at. The probabilities are $p_A$, $p_B$ and $p_C$ and we assume $p_A \leq p_B \leq p_C$.


They take turns shooting, first Alice, then Bob, the Carol, then back to Alice, and so on until one is left. What is Alice's best course of action?

We assume that all players act rationally. They will maximize their chance of survival.

**Basic idea**

1. To maximise their chances, each player prefer to be left with a weaker opponent.
2. So Bob would not shoot at Alice in preference to Carol, and Carol will not shoot at Alice in preference to Bob.  Carol and Bob will target each other until one dies as they represent the greatest threat to each other.
3. After one player dies, we are left with a two-way duel. Whoever shooting first will get an edge.
4. Alice would rather face Bob than Caral

Therefore, there are basically two competing guiding principles:
- **First shot advantage**: Since Alice is safe in a three-way duel, Alice can simply shoot into the air and let Bob and Carol shoot each other. This way, whenever B or C dies, A gets to shoot first at the remaining player.
- **Weaker opponent advantage**: Alice can shoot Carol to maximize the chance of facing Bob.

**Remark**

Most of the solution I found online simply focus solely on the **First Shot advantage**, and conclude **wrongly** that Alice should simply shoot into the air.

We will demonstrate this is only true in certain circumstances and in general, one needs to compare quantitatively two strategies.

# Two-way duel

Let’s solve a two way duel before going further. We will follow the notation in [@nigelcoldwell](https://puzzles.nigelcoldwell.co.uk/fiftyone.htm)

Let $p(\underline{X}, Y)$ be the probability of $X$ surviving a duel with $Y$ where $X$ has first shot

$$
p(\underline{X}, Y)=\operatorname{Pr}(X \text { hits } Y)+\operatorname{Pr}(X \text { misses } Y) * \operatorname{Pr}(Y \text { misses } X) * p(\underline{X}, Y)
$$

Let  $p(X, \underline{Y})$ be the probability of $X$ surviving a duel with $Y$ where $Y$ has first shot

$$
p(X, \underline{Y})=\operatorname{Pr}(Y\text{ misses } X) *(\operatorname{Pr}(X\text{ hits } Y)+\operatorname{Pr}(X\text{ misses }Y) * p(X, \underline{Y}))
$$

As a check, this is equal to $1-p(\underline{Y}, X)$ since only one person can survive a two person duel.

For example, we have the probability of A killing B if A shoots first
$$
p(\underline{A}, B) = \frac{p_A}{p_A+p_B - p_Ap_B}
$$
and if B shoots first
$$
p({A}, \underline{B}) = \frac{p_A}{p_A+p_B - p_Ap_B}(1-p_B)
$$

In [36]:
import pandas as pd
import numpy as np

def two_way_table(pA, pB):
    # probability of A winning if A shoots first
    A_first = pA/(1 - (1-pA) * (1-pB) )
    return A_first

In [37]:
# Probabilities for A, B, and C
pA = 1/3
pB = 2/3
pC = 1.0
print(f"p(A kills B, A first) = {two_way_table(pA, pB):.4f}")
print(f"p(A kills C, A first) = {two_way_table(pA, pC):.4f}")
print(f"p(A kills B, B first) = {1 - two_way_table(pB, pA):.4f}")
print(f"p(A kills C, C first) = {1 - two_way_table(pC, pB):.4f}")

p(A kills B, A first) = 0.4286
p(A kills C, A first) = 0.3333
p(A kills B, B first) = 0.1429
p(A kills C, C first) = 0.0000


# Three-way duel

Let's now compare two strategies.

1. Strategy I: Alice shoots into the air until either B or C dies.

$$
p_I(\text { Alice survive })=p(\underline{B}, C)* p(\underline{A}, B)+ (1-p(\underline{B}, C)) * p(\underline{A}, C)
$$

2. Strategy II: Alice shoots Carol and then face Bob.
$$
p_{II}(\text{ Alice survive }) = p_A * p(A, \underline{B}) + (1-p_A) * p_I(\text{ Alice survive })
$$

In [42]:
pA = 1/3
pB = 2/3
pC = 1

pI = two_way_table(pB, pC) * two_way_table(pA, pB) + (1 - two_way_table(pB, pC)) * two_way_table(pA, pC)
# print(f"p(A wins C, A first) = {two_way_table(pA, pC):.4f}")
# print(f"p(A wins B, B first) = {1 - two_way_table(pB, pA):.4f}")
print('if A shoots into the air,')
print(f"p_I(A survives) = {pI:.4f}")
print('if A shoots Carol and then face Bob,')
print(f"p_II(A survives) = {pA * (1 - two_way_table(pB, pA)) + (1 - pA) * pI:.4f}")

if A shoots into the air,
p_I(A survives) = 0.3968
if A shoots Carol and then face Bob,
p_II(A survives) = 0.3122


In this scenario, A should shoot into the air.

In [50]:
pA = 1/6
pB = 1/5
pC = 1

pI = two_way_table(pB, pC) * two_way_table(pA, pB) + (1 - two_way_table(pB, pC)) * two_way_table(pA, pC)
print('if A shoots into the air,')
print(f"p_I(A survives) = {pI:.4f}")
print('if A shoots Carol and then face Bob,')
print(f"p_II(A survives) = {pA * (1 - two_way_table(pB, pA)) + (1 - pA) * pI:.4f}")

if A shoots into the air,
p_I(A survives) = 0.2333
if A shoots Carol and then face Bob,
p_II(A survives) = 0.2611


**In this case, A should shoot C!**

# Simulations

This is adapted from [@nigelcoldwell](https://puzzles.nigelcoldwell.co.uk/fiftyone.htm)

In [49]:
import random
import time
from math import floor, pow

# Function to handle the truel simulation logic
def truel_function(pA, pB, pC, total_it_time, a_plan):
    # a_plan is "B", "C" or "Zero"


    if total_it_time < 1 or total_it_time != floor(total_it_time):
        print("total_it_time needs to be an integer, e.g. 5000")
        return

    if max(pA, pB, pC) > 1 or min(pA, pB, pC) < 0 or (pA + pB + pC) == 0:
        print("probability should be in [0, 1]")
        return


    totaltotalshots = 0
    max_shots = 0
    winsA = 0
    winsB = 0
    winsC = 0

    start_time = time.time()
    a = 0

    while time.time() - start_time < total_it_time / 1000.0:
        num_alive = 3
        a_alive = b_alive = c_alive = 1
        total_shots = 0

        while num_alive > 1:
            # A's turn to shoot
            if a_alive and num_alive > 1:
                target_a = -1
                if num_alive == 2:
                    target_a = 2 if b_alive else 3
                elif num_alive == 3:
                    if a_plan == "B":
                        target_a = 2
                    elif a_plan == "C":
                        target_a = 3
                    elif a_plan == "Zero":
                        target_a = 0

                if target_a == 2 and shoots(pA):
                    b_alive = 0
                    num_alive -= 1
                elif target_a == 3 and shoots(pA):
                    c_alive = 0
                    num_alive -= 1

                total_shots += 1

            # B's turn to shoot
            if b_alive and num_alive > 1:
                target_b = -1
                if num_alive == 2 and c_alive:
                    target_b = 3
                elif num_alive == 2 and a_alive:
                    target_b = 1
                elif num_alive == 3:
                    if pA > pC:
                        target_b = 1
                    elif pC > pA:
                        target_b = 3
                    else:
                        target_b = 1 + 2 * round(random.random())

                if target_b == 1 and shoots(pB):
                    a_alive = 0
                    num_alive -= 1
                elif target_b == 3 and shoots(pB):
                    c_alive = 0
                    num_alive -= 1

                total_shots += 1

            # C's turn to shoot
            if c_alive and num_alive > 1:
                target_c = -1
                if num_alive == 2 and b_alive:
                    target_c = 2
                elif num_alive == 2 and a_alive:
                    target_c = 1
                elif num_alive == 3:
                    if pA > pB:
                        target_c = 1
                    elif pB > pA:
                        target_c = 2
                    else:
                        target_c = 1 + round(random.random())

                if target_c == 1 and shoots(pC):
                    a_alive = 0
                    num_alive -= 1
                elif target_c == 2 and shoots(pC):
                    b_alive = 0
                    num_alive -= 1

                total_shots += 1

        winsA += a_alive
        winsB += b_alive
        winsC += c_alive
        totaltotalshots += total_shots
        max_shots = max(total_shots, max_shots)
        a += 1

        end_time = time.time()
        duration_time = round((end_time - start_time) * 1000, 3)
        average_shots = round(totaltotalshots / a, 3)

        print(f"Hit probabilities were A: {round(pA, 4)} B: {round(pB, 4)} C: {round(pC, 4)}")
        print(f"A was targeting {a_plan}")
        print("Survivals rates were:")
        print(f"A: {winsA} ({round(100 * winsA / a, 4)}%)")
        print(f"B: {winsB} ({round(100 * winsB / a, 4)}%)")
        print(f"C: {winsC} ({round(100 * winsC / a, 4)}%)")


def shoots(p):
    return random.random() <= p

print("Worst strategy")
truel_function(pA=1/3, pB=2/3, pC=1, total_it_time=10000, a_plan="B")
print("-"*40)
print("Strategy I: A shoots into air, until either B or C dies")
truel_function(pA=1/3, pB=2/3, pC=1, total_it_time=10000, a_plan="Zero")
print("-"*40)
print("Strategy II: A shoots C")
truel_function(pA=1/3, pB=2/3, pC=1, total_it_time=10000, a_plan="C")

Worst strategy
Hit probabilities were A: 0.3333 B: 0.6667 C: 1
A was targeting B
Survivals rates were:
A: 1064383 (26.4353%)
B: 1023639 (25.4234%)
C: 1938349 (48.1413%)
----------------------------------------
Strategy I: A shoots into air, until either B or C dies
Hit probabilities were A: 0.3333 B: 0.6667 C: 1
A was targeting Zero
Survivals rates were:
A: 1381893 (39.6851%)
B: 1326621 (38.0978%)
C: 773630 (22.2171%)
----------------------------------------
Strategy II: A shoots C
Hit probabilities were A: 0.3333 B: 0.6667 C: 1
A was targeting C
Survivals rates were:
A: 1078301 (31.2087%)
B: 1865498 (53.9921%)
C: 511330 (14.7992%)


In [51]:
print("Worst strategy")
truel_function(pA=1/6, pB=1/5, pC=1, total_it_time=10000, a_plan="B")
print("-"*40)
print("Strategy I: A shoots into air, until either B or C dies")
truel_function(pA=1/6, pB=1/5, pC=1, total_it_time=10000, a_plan="Zero")
print("-"*40)
print("Strategy II: A shoots C")
truel_function(pA=1/6, pB=1/5, pC=1, total_it_time=10000, a_plan="C")

Worst strategy
Hit probabilities were A: 0.1667 B: 0.2 C: 1
A was targeting B
Survivals rates were:
A: 583394 (19.4495%)
B: 249311 (8.3117%)
C: 2166823 (72.2388%)
----------------------------------------
Strategy I: A shoots into air, until either B or C dies
Hit probabilities were A: 0.1667 B: 0.2 C: 1
A was targeting Zero
Survivals rates were:
A: 685895 (23.333%)
B: 294212 (10.0086%)
C: 1959487 (66.6584%)
----------------------------------------
Strategy II: A shoots C
Hit probabilities were A: 0.1667 B: 0.2 C: 1
A was targeting C
Survivals rates were:
A: 749736 (26.0923%)
B: 527076 (18.3433%)
C: 1596589 (55.5644%)
